In [3]:
import pandas as pd

In [18]:
pd.set_option('display.max_colwidth', -1)

,test,oui
0,2,3
1,3,4


# Load pickle files

In [29]:
allocine_reviews = {}
pickle_files = [
    "pickle/allocine_1_10.pickle",
    "pickle/allocine_11_20.pickle",
]

data = pd.DataFrame()

for path in pickle_files:
    # For pickle file in pickle folder :
    with open(path, 'rb') as handle:
        df = pd.read_pickle(path)
        data = data.append(df, ignore_index=True)        

In [30]:
data

,film-id,rating,review,date,helpful,unhelpful
0,1,1.0,"Je l’ai déjà dit plusieurs fois sur ce site, mais je le redis encore une fois: Jack Nicholson, j’adore, tout simplement. C’est comme ça depuis je ne sais combien de temps, et ça ne changera jamais, soyez-en sûrs. Et c’est justement pour ça, que c’est avec vraiment beaucoup de peine que je n’ai collé qu’une petite étoile à ce film. Je plaisante pas, ça m’a vraiment pas fait plaisir, mais, pour le coup, j’ai pas eu d’autres solutions étant donné que ce « Facteur sonne toujours deux fois » ne m’a pas du tout inspiré. Pourtant, Rafelson/Nicholson, ça avait plutôt bien fonctionné par le passé. Rappelons nous du très bon « Cinq pièces faciles » de 1970... Mais là rien, vraiment rien. Pas une seule seconde je n’ai réussi à m’intéresser au film. J’ai trouvé ça lent et surtout très insipide. Mais je me disais que ça allait finir par se lancer à un moment ou un autre, mais non, on reste au point mort tout le temps. Et l’on s’ennuie quand même pas mal. Même le grand Jack ne parvient pas à tirer tout cela un peu vers le haut ! Jessica Lange, quant à elle, est encore une fois difficilement supportable. C’est le premier raté du duo Rafelson/Nicholson. Le deuxième étant le catastrophique « Man Trouble ».",2017-10-07,3,0
1,1,4.0,"La prestation sans faute de Jack Nicholson et celle stupéfiante et débordante de sensualité de Jessica Lange font que ce film est essentiellement un film d'acteur. La partie judiciaire de l'intrigue est malheureusement ratée car peu compréhensible (les ""faux"" aveux, la négociation sur la prime, le jugement de Clara... tout cela est extrêmement confus). En revanche, on appréciera la courte irruption quasi surréaliste de la très troublante Angelica Huston en dompteuse de fauves. Et puis il y a cet érotisme très subtil que dégage le film d'un bout à l'autre, pratiquement rien n'est montré mais on n'est pas près d'oublier la petite nuisette quasi transparente de Jessica Lange. Il est simplement dommage que la prod n'est pas choisi un réalisateur de renom, sinon les ingrédients pour en faire un chef d'œuvre étaient bien là.",2014-04-23,5,1
2,1,4.0,"De la dynamite! Les annèes 80 ont ètè marquèes par un retour au thème et aux personnages du passè! Ce retour au passè peut aussi permettre une vèritable approche de certaines thèmes, jusque là occultès par la censure! C'est ainsi que ""The Postman Always Rings Twice"" de Bob Rafelson peut enfin aller jusqu'au bout des implications du formidable roman de James Crain et donner à la sexualitè la place qui lui revient enfin dans l'histoire. ""Body Heat"" de Kasdan où, comme cette quatrième version de ""The Postman Always Rings Twice"" après Pierre Chenal, Luchino Visconti et Tay Garnett (ainsi que la plupart des romans de Cain), c'est la femme qui mène le jeu! Et elle mène le jeu avec ses armes! Dèfinitivement affranchie sexuellement, elle joue de son corps en toute libertè et sans ressentir la moindre culpabilitè! L'imposant Jack Nicholson et la sublime Jessica Lange forment un couple extraordinaire et leurs èbats sulfureux dans la farine ont fait date dans l'histoire du cinèma! Une scène de cul grand cru avec deux excellents comèdiens qui se sont totalement investis dans la conception de cette oeuvre intense et sensuelle, prototype même du film noir contemporain de demain! Tous les ingrèdients sont rèunis pour un duo de lègende à jamais dans nos mèmoires! Et dire que certaines critiques ont jugè ce remake racoleur et superficiel! Comme quoi, tous les goûts cinèphiliques sont dans la nature même si selon moi la version en noir & blanc de Garnett est un cran au-dessus à la version colorisèe de Rafelson...",2014-09-10,4,3
3,1,4.0,"Quatrième et dernière adaptation en date de la célèbre oeuvre de James M. Cain, cette version joue surtout sur la passion qui unit les deux amants pour faire avancer l'intrigue. Ce qui donne lieu à quelques scènes d'une sensualité exacerbée notamment celle où les deux personnages deviennent amants. O

In [25]:
print("Un bon film d'aventure familial, porté par l'énergie et le charisme de Belmondo.\nL'humour y est très présent, en tournant en dérision toute la bêtise de l'idéologie nazi.\nIl y a aussi d’impressionnantes séquences d'action; dont une (les héros pourchassés par des nazis sur des motos) que Steven Spielberg a peut-être vu avant de signer le chef-d'oeuvre \"Indiana Jones et la dernière croisade\".")

Un bon film d'aventure familial, porté par l'énergie et le charisme de Belmondo.
L'humour y est très présent, en tournant en dérision toute la bêtise de l'idéologie nazi.
Il y a aussi d’impressionnantes séquences d'action; dont une (les héros pourchassés par des nazis sur des motos) que Steven Spielberg a peut-être vu avant de signer le chef-d'oeuvre "Indiana Jones et la dernière croisade".
